In [1]:
import os
os.chdir("/content/drive/MyDrive/Colab/tensorflow_&_pytorch_classifier")

In [2]:
from plot_model import PlotModel
from customization import Customize , LR_ASK
from make_model import MakeModel
import os
import pandas as pd
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

def run():
    Customizing = Customize(framework = 'tensorflow')
    msg = 'enter the full path to the working directory where data will be stored.'
    working_dir=input(msg)
    if 'kaggle' in working_dir:  # if running the notebook on kaggle need to modify so parameters
        delimiter = '/'
    else:
        delimiter = '\\'
    msg = 'Enter the full path to the train directory'
    train_dir =input(msg)
    msg = 'Enter the full path to the validation directory. If there is no validation directory press enter'
    ans = input(msg)
    if ans == '':
      val_dir = None
    else:
        valid_dir = ans
    msg = 'Enter the full path to the test directory. If there is no test directory press enter'
    ans = input(msg)
    if ans == '':
      test_dir = None
    else:
      test_dir = ans
    ftotal, flargest, maxclass, fsmallest, minclass = Customizing.check_dataset_size(train_dir)  # get info on training data
    msg1 = f'the train directory contains {ftotal} files, class {maxclass} has the most images of {flargest} files\n'
    msg2 = f'class {minclass} contains the least number of images of {fsmallest} files\n'
    msg3 = f'NOTE if the value of most images is the same as the value of least images the dataset is balanced\n'
    msg4 = 'When dealing with very large data sets to save time you may not want to read in all the image files\n'
    msg5 = 'to limit the maximum number of image files in any class you can enter an integer limiter value'
    msg = msg1 + msg2 + msg3 + msg4 + msg5
    PlotModel.print_in_color(msg)
    msg = 'input a limiter integer value to limit max number of images in a class, for no limit hit enter'
    limiter = input(msg)
    if limiter == '':
      limiter = None
    else:
      limiter = int(limiter)
      msg = f'images will be limited to a maximum of {limiter} images in each class'
      PlotModel.print_in_color(msg)
    # create train, test and valid data frames
    train_df, test_df, valid_df, classes, class_count, max_samples, min_samples, have, wave = Customizing.make_data_frames(train_dir , test_dir , val_dir  , limiter )
    msg = f'the average height of training images is {have} and average width is {wave}\n  enter the image height to be used to train the model'
    img_height = int(input(msg))
    msg = ' Enter the image width to be used to train the model'
    img_width = int(input(msg))
    img_size = (img_height, img_width)
    msg = f'model will be trained with image shape of  ( {img_height}, {img_width} )'
    PlotModel.print_in_color(msg)
    # determine if user wants to balance the training dataframe
    train_df, img_size, classes, class_count = Customizing.preprocess_dataset(train_df, img_size, max_samples, min_samples,
                                                                  working_dir)
    # make train, test and valid data generators
    train_gen, test_gen, valid_gen, test_steps, class_count = Customizing.make_gens(20, 'labels', train_df, test_df, valid_df,
                                                                        img_size)
    PlotModel.show_image_samples(train_gen)  # show some sample training images
    msg = 'select model size enter S for small, L for large or hit enter (recommended) for medium size model'
    ans = input(msg)
    model = MakeModel.make_model(img_size, class_count, .001, ans)  # create the model
    epochs = 100
    ask_epoch = 10
    ask = LR_ASK(model, epochs=epochs, ask_epoch=ask_epoch)  # instantiate the custom callback
    callbacks = [ask]
    msg = f'Your model will start training. After {ask_epoch} epochs you will be asked if you wish to halt training by entering H\nor enter the number of additional epochs to run, then you will be queired again after those epochs complete.\n To decide look at the % decrease in validation loss if it is less than 2% enter H to halt training\n Note your model is set with the weights from the epoch with lowest validation loss'
    PlotModel.print_in_color(msg)
    history = model.fit(x=train_gen, epochs=epochs, verbose=1, callbacks=callbacks, validation_data=valid_gen,
                        validation_steps=None, shuffle=True, initial_epoch=0)  # train the model
    PlotModel.tr_plot(history)  # plot training data
    msg = 'To save the training data to a csv file enter the name for the csv file or press enter to not save the data'
    ans = input(msg)
    if ans != '':  # same the training data to a csv file history.csv in working directory
        csvpath = os.path.join(working_dir, ans + '.csv')
        Customizing.save_history_to_csv(history, csvpath)
        msg = f'training data saved to {csvpath}'
        PlotModel.print_in_color(msg)
    # do predictions on test set and generate reports
    errors, tests, error_list, error_pred_list, f1score = Customizing.predictor(model,test_gen)
    Customizing.print_errors(error_list, error_pred_list, delimiter)
    msg = f'your trained model will be saved to directory {working_dir} enter a subject for the saved model'
    subject = input(msg)
    MakeModel.save_model(model, subject, classes, img_size, f1score, working_dir)  # save the model to the working directory
    msg = 'model save nomenclature is directory/subject-number of classes- (img_height, img_width)- F1score.h5'
    PlotModel.print_in_color(msg)
    msg = 'to run the classifier again enter R, to halt press enter  '
    ans = input(msg)
    if ans == 'R' or ans == 'r':
        run()
    else:
        PlotModel.print_in_color('Process Complete')
    return error_list, error_pred_list

if __name__ == '__main__':
    error_list, error_pred_list = run()

Output hidden; open in https://colab.research.google.com to view.